# Cross-Validation
In most real-word applications we do not know the data universe, i.e. we do not know all possible data points that might be there. Our training data is possibly just a biased subsample of the population.<br>
When we fit our algorithm to such a subsample its performance will degrade, when applied to new, unseen data points. In order to have an idea, how well our algorithm will perform in such cases, we can use a cross-validation scheme:<br>
In the example below, a 5-fold cross-validation is illustrated.
* split the training data in 5 equal sized parts. In sklearn you can choose StratifiedKFold, what essentially tries to keep the percentage of all classes within the folds.
* train your algorithmm on 4 folds and classify data in the 5th hold-out fold. Keep the performance on this fold.
* repeat the last step 4 more times and use each time another fold as your hold-out fold.
* at the end, you have 5 independent estimates of your algorithm's performance
* compute the mean of theses 5 estimates for an overall estimate

<img alt="generated with latex" caption="The gradient boosting algorithm" id="gradient_boosting" src="../images/cross_validation.png">

### Pseudo python code:

In [ ]:
# this is pseudo-code...algorithm and data are not defined here
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

five_fold = StratifiedKFold(y=y_train, n_folds=5, shuffle=True)

auc_scores = []
for train_idx, val_idx in five_fold: 
    algorithm.fit(X_train[train_idx], y_train[train_idx])
    prediction = algorithm.predict(X_train[val_idx])
    auc_scores.append(roc_auc_score(y_true = y_train[val_idx], y_score = prediction))
    
print(f'mean area under the curve: {np.mean(auc_scores)}')    

NameError: name 'y_train' is not defined

# Nested cross-validation
We learned how to estimate the true performance of an algorithm. Now, suppose we have to find the best parameters for an algorithm. We use a grid search and use for each setting the cross-validation scheme as described. We then use the best parameter combination and report the estimate of the true performance on unseen data.<br>
Here, we introduce a bias, because we do an extensive search over different settings and in the end get the parameters with best cross-validation result on the training data. In other words, we probably fitted to peciuliarities of the training data. The right way to do it is to use the trainings-folds as well for the parameter search. We then evaluate on the hold-out fold only once. This is called nested cross-validation because the cross-validation scheme for finding the best parameters is nested within our cross-validation that aims to get a unbiased estimate of the algorithm's performance:
<img alt="generated with latex" caption="The gradient boosting algorithm" id="gradient_boosting" src="../images/nested_cross_validation.png">

# Stacking
Stacking is a technique that is related to cross-validation methods. It became very famous for kaggle competitions. Now, it is exhaustively used in every competition.<br>
Imagine, you think of using the output of one algorithm as an input to just another algorithm. If you fit the first algorithm to all the trainings data, the algorithm will certainly overfit. The generalization to unseen test data or to productive data will be bad. But, as we learned, we can mitigate these problems by the use of a cross-validation scheme. Stacking uses cross-validation for scoring the trainings data and algorithm ensembling for the test data. The important thing here to remember is, that the scores in the training data are __independent__. __The part of the training data scored in each iteration is not part of the data used to fit the algorithm__.
<img alt="generated with latex" caption="The gradient boosting algorithm" id="gradient_boosting" src="../images/stacking.png">